# Семинар 3. Исправление опечаток

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report

Возьмем данные с соревнования Dialog Evaluation 2015 по исправлению опечаток. Данные представляют собой набор предложений (правильное - ошибочное). Задача найти слова с ошибками и заменить их на правильный вариант.

Недостатком тут является то, что не всегда можно правильно сопоставить слова правильного предложения и ошибочного (из-за слов с пропущенным или добавленным пробелом). Из статьи авторов корпуса не очень понятно, как они решали эту проблему, поэтому я просто удалил все такие предложения.

In [2]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [3]:
# Посмотрим на пары предложений
print(bad[2])
print(true[2])

Пояним эту мысль.
Поясним эту мысль


In [4]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [5]:
pprint(align_words(true[1], bad[1]))

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]


Вытащим только неправильные варианты и заодно посчитаем процент ошибок.

In [6]:
mistakes = []
total = 0

for i in range(len(true)):
    
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        
        total += 1

In [7]:
print('Доля ошибок - ', len(mistakes)/total )

Доля ошибок -  0.13034358769476628


In [8]:
mistakes[:5]

[('симпатичнейшее', 'симпатичнейшое'),
 ('апофеозом', 'опофеозом'),
 ('поясним', 'пояним'),
 ('получатся', 'полчатся'),
 ('очень', 'оччччень')]

Обернем в Counter, чтобы сразу увидеть частотные ошибки.

In [9]:
Counter(mistakes).most_common(10)

[(('сегодня', 'седня'), 24),
 (('вообще', 'вобще'), 18),
 (('вообще', 'ваще'), 17),
 (('естественно', 'естесственно'), 17),
 (('хочется', 'хочеться'), 16),
 (('кстати', 'кстате'), 16),
 (('очень', 'ооочень'), 14),
 (('как-то', 'както'), 9),
 (('очень', 'оооочень'), 9),
 (('это', 'ето'), 9)]

Из-за того, что процент ошибок довольно низкий, не очень выгодно будет находить исправление для каждого слова. Нужен какой-то более простой классификатор, который выделит ошибочные слова, чтобы потом только их и редактировать.

Самый простой способ это сделать - составить словарь правильных слов и потом сравнивать с ним. Чтобы не делать этого вручную, можно взять какой-нибудь корпус текстов, прошедщих редактуру. Новостные тексты для этого хорошо подходят.

Я заранее собрал небольшой корпус новостных текстов и немного почистил его удалив отдельную пунктуацию и пунктуацию на границах слов.

In [10]:
corpus = open('corpus_ng.txt', encoding='utf8').read().splitlines()

In [11]:
# нормализация нам тут не нужна так как нужно находить слова в разных формах
print(corpus[1].split()[:10])

['судя', 'по', 'всему', 'русская', 'православная', 'церковь', 'нашла', 'долгожданную', 'национальную', 'идею']


## Задание 1.
Напишите функцию, которая будет предсказывать ошибочные слова на основе корпуса.

In [12]:
corpus[2]

'буду краток одиссей хоть и был туристом но никогда не писал на скалах фразы вроде здесь был одя потому что за такие художества морской бог посейдон мог и трезубцем заехать а одиссей не оскорблял чувства богов и верующих в них граждан губернатор алтайского края александр карлин согласен с греческим героем местные энтузиасты даже реализуют проект чистые скалы  достояние алтая очищая горы от надписей а то многие как остап бендер всюду норовят оставить память о себе в виде надписей типа киса и ося здесь были губернатор карлин не разделяет подобных порывов а про своего коллегу не называя его рассказал следующую историю был александр богданович с женой в карловых варах и там рассказывает губернатор есть очень достойный памятник  он посвящен членам правящей династии романовых которые побывали в этих местах я читаю все имена продолжает карлин и ниже списка  фамилию моего коллеги он не из царской семьи у него простая фамилия и исполнено это чуть-чуть посолиднее чем здесь был вася карлин по его

In [13]:
# создайте множество, чтобы проверять вхождения
vocab = set()

## ваш код здесь
for body in corpus:
    newset = set(body.split())
    vocab = vocab|newset

In [14]:
def predict_mistaken(word, vocab):
    '''
    ::input: word, vocabulary
    ::output: 1 or 0
    
    '''
    ## ваш код здесь
    return 1 * (word not in vocab)
    

In [15]:
vocab

{'выучат',
 'отныне',
 'сословия',
 'австрийской',
 'вдохновенный',
 '2015-м',
 'набравшей',
 'инцидентах',
 'неправ',
 'недовыработка',
 'номиналу',
 'накладные',
 'планирования',
 'уронившие',
 'меняющих',
 'интересовала',
 'сбегал',
 'кривулина',
 'общалась',
 'резкую',
 'предложено',
 'компьютеров',
 '554',
 'ввт',
 'облегчение',
 'отложено',
 'небывалые',
 'встретятся',
 'боголепова',
 'спинку',
 'пагубной',
 'продления',
 'округ',
 'пигментных',
 'ловли',
 'белобандитов',
 'обособленности',
 'выделяем',
 'иранцами',
 'самозапись',
 'вышке',
 'engagement',
 'норвегией',
 'окаменевшем',
 'маститых',
 'домогательств',
 'тарасова',
 'иллюстраций',
 'подотраслях',
 'известнейших',
 'пакт',
 'металлист',
 'слетел',
 'спэни',
 'мосфильма',
 'ненавистью',
 'проникшись',
 'зеллнеры',
 'комплиментом',
 'дисперсно',
 'дополнить',
 'бишкеке',
 'знамен',
 'предпринимательская',
 'рассматривалась',
 'юго-западному',
 'каширке',
 'матушке',
 'ата-мекен',
 'внимающая',
 'паузой',
 'проверю',
 'с

In [16]:
# для оценки создайте два списка y_true и y_pred
# пройдитесь по предложениям
# сопоставите слова с помощью функции align_words
# пройдитесь по парам слов и
# если слова одинаковые добавьте в y_true 0 
# если слова разные добавьте в y_true 1
# предскажите ошибочность слова из bad списка 
# добавьте предсказание в список y_pred

y_true = []
y_pred = []

## ваш код здесь
for correct, wrong in zip(true, bad):
    pairs = align_words(correct, wrong)
    
    for correct_word, wrong_word in pairs:
        
        if correct_word == wrong_word:
            y_true.append(0)
        else:
            y_true.append(1)
        
        pred = predict_mistaken(wrong_word, vocab)
        y_pred.append(pred)


In [17]:
# оцените качество с помощью classification_report
print(classification_report(y_true, y_pred))

             precision    recall  f1-score   support

          0       0.98      0.89      0.93      8707
          1       0.55      0.88      0.68      1305

avg / total       0.92      0.89      0.90     10012



### Генерация исправлений

Теперь нужно думать о том, как исправить неправильные слова. Посмотрим как это можно делать на примере известной реализации Питера Норвига.

Основная идея - сделать словарь правильных слов (у нас уже есть), расчитать вероятность каждого слова в корпусе.

In [18]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [19]:
WORDS.most_common(10)

[('в', 67679),
 ('и', 55933),
 ('на', 27860),
 ('не', 21627),
 ('что', 18299),
 ('с', 18224),
 ('по', 13117),
 ('а', 9696),
 ('как', 8958),
 ('к', 8907)]

In [20]:
# фунцкия расчета вероятности слова
N = sum(WORDS.values())
def P(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N


In [21]:
P('апофеоз')

1.8025455548325345e-06

Чтобы найти исправления нужно сгенерировать возможные исправления и выбрать те, которые есть в словаре. Если есть несколько вариантов, то выбрать тот, у котогоро наибольшая вероятность.

In [22]:
def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [23]:
%%time
correction('слива')

Wall time: 0 ns


'слова'

Давайте поподробнее разберем, что происходит в функции edits.

In [24]:
word = 'опефеоз'
splits = [(word[:i], word[i:])    for i in range(len(word) + 1)]

In [25]:
splits[:10]

[('', 'опефеоз'),
 ('о', 'пефеоз'),
 ('оп', 'ефеоз'),
 ('опе', 'феоз'),
 ('опеф', 'еоз'),
 ('опефе', 'оз'),
 ('опефео', 'з'),
 ('опефеоз', '')]

In [26]:
deletes = [L + R[1:] for L, R in splits if R]

In [27]:
deletes[:10]

['пефеоз', 'оефеоз', 'опфеоз', 'опееоз', 'опефоз', 'опефез', 'опефео']

In [28]:
transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]

In [29]:
transposes[:10]

['поефеоз', 'оепфеоз', 'опфееоз', 'опеефоз', 'опефоез', 'опефезо']

In [30]:
letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
replaces = [L + c + R[1:] for L, R in splits if R for c in letters]

In [31]:
len(replaces)

231

In [32]:
inserts = [L + c + R for L, R in splits for c in letters]

In [33]:
inserts[:10]

['йопефеоз',
 'цопефеоз',
 'уопефеоз',
 'копефеоз',
 'еопефеоз',
 'нопефеоз',
 'гопефеоз',
 'шопефеоз',
 'щопефеоз',
 'зопефеоз']

Для оценки используем просто долю правильных исправлений.

In [34]:
# До этого мы уже считали долю ошибок во всех предложениях.
# Поэтому если ничего не менять то доля правильных исправлений уже будет 100 - 13 = 87 %.
# Наш подход соответственно должен показывать лучший результат 
correct = 0
total = 0

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        
        predicted = correction(pair[1])
        if predicted == pair[0]:
            correct += 1
        total += 1
    if not i % 10:
        print(i)
        print(correct/total)

0
0.7333333333333333


KeyboardInterrupt: 

In [35]:
print(correct/total)

0.8620689655172413


In [36]:
%%time
correction('нав')

Wall time: 0 ns


'на'

In [37]:
%%time
correction('насмехатьсяаававттававаываываы')

Wall time: 5.56 s


'насмехатьсяаававттававаываываы'

Посмотрим какие исправления выбираются для самых частотных опечаток.

In [38]:
[(wt[0], wt[1], correction(wt[1])) for wt, _ in Counter(mistakes).most_common(10)]

[('сегодня', 'седня', 'сеня'),
 ('вообще', 'вобще', 'вообще'),
 ('вообще', 'ваще', 'ваще'),
 ('естественно', 'естесственно', 'естественно'),
 ('хочется', 'хочеться', 'хочется'),
 ('кстати', 'кстате', 'кстати'),
 ('очень', 'ооочень', 'очень'),
 ('как-то', 'както', 'какао'),
 ('очень', 'оооочень', 'оооочень'),
 ('это', 'ето', 'ето')]

### Метрики близости слов.

Вместо того, чтобы генерировать все варианты, можно искать похожие слова в словаре. Для этого нужно задать метрику похожести. Для исправления опечаток часто используются расстояния редактирования (количество редактирований, которые нужно сделать в строке a, чтобы прийти к b.

In [39]:
# в питоне есть библиотеке для нахождения близких строк

In [40]:
from difflib import get_close_matches

In [41]:
%%time
get_close_matches('опофеоз', WORDS.keys(), n=1)

Wall time: 1.42 s


['апофеоз']

Работает тоже не очень быстро.

Недавно вышла библиотека, в которой реализованы многие методы нахождения расстояний.

In [42]:
!pip install textdistance

You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [43]:
import textdistance


In [44]:
def get_closest_match_with_metric(text, lookup, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(1)[0]

In [45]:
%%time
get_closest_match_with_metric('опофиоз', WORDS, textdistance.hamming)

Wall time: 5.66 s


('апофеоз', 0.7142857142857143)

In [46]:
%%time
get_closest_match_with_metric('апофиоз', WORDS, textdistance.levenshtein)

Wall time: 1min 2s


('апофеоз', 0.8571428571428572)

Ждать так долго мы не можем, поэтому попробуем что-то побыстрее.

Многие вещи, которые медленно решаются в питоне, можно оптимизировать с помощью матричных и векторных операций.

Сделаем поиск похожих по векторам символов, из которых состоит слово.

In [47]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [48]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [49]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [52]:
def get_closest_match_vec(text, X, vec, TOPN=10):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    
    return [id2word[top] for top in topn]

In [53]:
%%time
get_closest_match_vec('опофеоз', X, vec)

Wall time: 63 ms


['апофеозом',
 'апофеоз',
 'шизофреноподобное',
 'полезного',
 'позволено',
 'профсоюзного',
 'позором',
 'золотое',
 'розовое',
 'половое']

## Задание 2. 


Напишите функцию, которая принимает слово и находит ближайшее к нему в словаре (сгенерируйте несколько кандидатов с помощью get_closest_match_vec, а затем посчитайте метрики близости до кадого слова и выведете самое близкое). 

In [58]:
def get_closest_hybrid_match(text, X, vec, metric=textdistance.levenshtein):
    Suggestions = get_closest_match_vec(text, X, vec, TOPN=10)
    closest = get_closest_match_with_metric(text, Suggestions, textdistance.levenshtein)[0]
    
    
    return closest

In [72]:
get_closest_hybrid_match('алкогнль', X, vec)

'алкоголь'

In [77]:
%%time

# Если ничего не менять, то доля правильных исправлений уже будет 100 - 13 = 87 %.
# Старый подход
correct = 0
total = 0

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        
        predicted = correction(pair[1])
        if predicted == pair[0]:
            correct += 1
        total += 1
    if not i % 100:
        print(i)
        print(correct/total)

0
0.7333333333333333
100
0.8512544802867383
200
0.8581048581048581
300
0.859375
400
0.8619809788912085
500
0.8610698365527489
600
0.8622976098689283
700
0.8622047244094488
800
0.8594993655554274
900
0.8596669374492283
Wall time: 5min 23s


In [83]:
print("Resulting method accuracy:", "{:.2f}".format(100*correct/total)+"%")

Resulting method accuracy: 86.02%


In [85]:
%%time

# Если ничего не менять, то доля правильных исправлений уже будет 100 - 13 = 87 %.
# Новый подход
new_correct = 0
new_total = 0

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        
        predicted = get_closest_hybrid_match(pair[1], X, vec)
        if predicted == pair[0]:
            new_correct += 1
        new_total += 1
    if not i % 100:
        print(i)
        print(new_correct/new_total)

0
0.5333333333333333
100
0.8243727598566308
200
0.8316498316498316
300
0.8347355769230769
400
0.8366968220830434
500
0.8380386329866271
600
0.8390131071703932
700
0.8385826771653543
800
0.8352751182373976
900
0.8362103980503656
Wall time: 11min 3s


In [86]:
print("Resulting method accuracy:", "{:.2f}".format(100*new_correct/new_total)+"%")

Resulting method accuracy: 83.66%


В рассмотренных методах при выборе исправления никак не использовался контекст. Про то как это можно сделать, можно почитать вот тут - https://habr.com/ru/post/346618/

In [87]:
# method enhansion is yet to come